# Guided Capstone Step 4. Pre-Processing and Training Data Development

**The Data Science Method**  


1.   Problem Identification 


2.   Data Wrangling 
  
 
3.   Exploratory Data Analysis   

4.   **Pre-processing and Training Data Development**  
 * Create dummy or indicator features for categorical variables
  * Standardize the magnitude of numeric features
  * Split into testing and training datasets
  * Apply scaler to the testing set
5.   Modeling 
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

**<font color='teal'> Start by loading the necessary packages as we did in step 3 and printing out our current working directory just to confirm we are in the correct project directory. </font>**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os 
from scipy.stats import linregress
import seaborn as sns

**<font color='teal'>  Load the csv file you created in step 3, remember it should be saved inside your data subfolder and print the first five rows.</font>**

In [2]:
os.getcwd()

'C:\\Users\\Bauyrjan.Jyenis'

In [3]:
path="C:\\Users\\Bauyrjan.Jyenis\\GuidedCapstone\\data"
os.chdir(path)

In [4]:
os.getcwd()

'C:\\Users\\Bauyrjan.Jyenis\\GuidedCapstone\\data'

In [5]:
os.listdir()

['clean_data.csv',
 'GuidedCapstone',
 'step2_output.csv',
 'step2_output_v1.0.csv',
 'step3_output.csv',
 'updated_ski_data.csv']

In [6]:
df = pd.read_csv('step3_output.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,Name,state,summit_elev,vertical_drop,trams,fastEight,fastSixes,fastQuads,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
0,2,2,Hilltop Ski Area,Alaska,2090,294,0,0.0,0,0,...,30.0,30.0,150.0,36.0,69.0,30.0,34.00000,152.0,30.0,2
1,4,4,Sunrise Park Resort,Arizona,11100,1800,0,0.0,0,1,...,800.0,80.0,115.0,49.0,250.0,74.0,78.00000,104.0,80.0,1
2,5,5,Yosemite Ski & Snowboard Area,California,7800,600,0,0.0,0,0,...,88.0,100.0,110.0,84.0,300.0,47.0,47.00000,107.0,0.0,1
3,8,8,Boreal Mountain Resort,California,7700,500,0,0.0,0,1,...,380.0,200.0,150.0,54.0,400.0,49.0,64.16681,150.0,200.0,1
4,9,9,Dodge Ridge,California,8200,1600,0,0.0,0,0,...,862.0,100.0,114.0,69.0,350.0,78.0,78.00000,140.0,0.0,1


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         173 non-null    int64  
 1   Unnamed: 0.1       173 non-null    int64  
 2   Name               173 non-null    object 
 3   state              173 non-null    object 
 4   summit_elev        173 non-null    int64  
 5   vertical_drop      173 non-null    int64  
 6   trams              173 non-null    int64  
 7   fastEight          173 non-null    float64
 8   fastSixes          173 non-null    int64  
 9   fastQuads          173 non-null    int64  
 10  quad               173 non-null    int64  
 11  triple             173 non-null    int64  
 12  double             173 non-null    int64  
 13  surface            173 non-null    int64  
 14  total_chairs       173 non-null    int64  
 15  Runs               173 non-null    float64
 16  TerrainParks       173 non

## Create dummy features for categorical variables

**<font color='teal'> Create dummy variables for `state`. Add the dummies back to the dataframe and remove the original column for `state`. </font>**

Hint: you can see an example of how to execute this in Aiden's article on preprocessing [here](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-pre-processing-and-training-data-development-fd2d75182967). 

In [8]:
dfo = pd.DataFrame(df['state'])
df = pd.concat([df.drop(dfo, axis=1), pd.get_dummies(dfo)], axis=1)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 62 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            173 non-null    int64  
 1   Unnamed: 0.1          173 non-null    int64  
 2   Name                  173 non-null    object 
 3   summit_elev           173 non-null    int64  
 4   vertical_drop         173 non-null    int64  
 5   trams                 173 non-null    int64  
 6   fastEight             173 non-null    float64
 7   fastSixes             173 non-null    int64  
 8   fastQuads             173 non-null    int64  
 9   quad                  173 non-null    int64  
 10  triple                173 non-null    int64  
 11  double                173 non-null    int64  
 12  surface               173 non-null    int64  
 13  total_chairs          173 non-null    int64  
 14  Runs                  173 non-null    float64
 15  TerrainParks          1

## Standardize the magnitude of numeric features

**<font color='teal'> Using sklearn preprocessing standardize the scale of the features of the dataframe except the name of the resort which we done't need in the dataframe for modeling, so it can be droppped here as well. Also, we want to hold out our response variable(s) so we can have their true values available for model performance review. Let's set `AdultWeekend` to the y variable as our response for scaling and modeling. Later we will go back and consider the `AdultWeekday`, `dayOpenLastYear`, and `projectedDaysOpen`. For now leave them in the development dataframe. </font>**

In [10]:
# first we import the preprocessing package from the sklearn library
from sklearn import preprocessing

# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X = df.drop(['Name','AdultWeekend'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y = df['AdultWeekend'] 

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = preprocessing.StandardScaler().fit(X)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled=scaler.transform(X) 

## Split into training and testing datasets

**<font color='teal'> Using sklearn model selection import train_test_split, and create a 75/25 split with the y = `AdultWeekend`. We will start by using the adult weekend ticket price as our response variable for modeling.</font>**

In [11]:
# Import the train_test_split function from the sklearn.model_selection utility.  
from sklearn.model_selection import train_test_split

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)

Here we start the actual modeling work. First let's fit a multiple linear regression model to predict the `AdultWeekend` price.

# Guided Capstone Step 5. Modeling


This is the fifth step in the Data Science Method. In the previous steps you cleaned and prepared the datasets. Now it's time to get into the most exciting part: modeling! In this exercise, you'll build three different models and compare each model's performance. In the end, you'll choose the best model for demonstrating insights to Big Mountain management.



### **The Data Science Method**  


1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   Pre-processing and Training Data Development

5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

## Fit Models with a Training Dataset

**<font color='teal'> Using sklearn, fit the model on your training dataset.</font>**

#### Model 1

In [12]:
# all first model set
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error
lm = linear_model.LinearRegression()
model_1 = lm.fit(X_train,y_train)

**<font color='teal'> Predict on the testing dataset and score the model performance with the y_test set and the y-pred values. The explained variance is a measure of the variation explained by the model. This is also known as the R-squared value. </font>**

Hint: you will have to use the `predict()` method here as it's used in this [DSM article](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-modeling-56b4233cad1b) about modeling. 

In [13]:
# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
y_pred = model_1.predict(X_test)

## Review Model Outcomes — Iterate over additional models as needed

In [14]:
# You might want to use the explained_variance_score() and mean_absolute_error() metrics.
# To do so, you will need to import them from sklearn.metrics. 
# You can plug y_test and y_pred into the functions to evaluate the model

In [15]:
evs=explained_variance_score(y_test, y_pred)

In [16]:
evs

-6.001832694550452e+27

In [17]:
mae=mean_absolute_error(y_test, y_pred)

In [18]:
mae

288582161650987.9

In [19]:
mae/df['AdultWeekend'].mean()

5073026511847.549

**<font color='teal'> Print the intercept value from the linear model. </font>**

Hint: our linear regression model `lm` has an attribute `intercept_` for the intercept

In [20]:
model_1.intercept_

-73396619148169.34

In [21]:
lm.intercept_

-73396619148169.34

In [22]:
df['AdultWeekend'].describe()

count    173.000000
mean      56.885601
std       15.373290
min       20.000000
25%       45.000000
50%       55.000000
75%       65.000000
max       93.000000
Name: AdultWeekend, dtype: float64

**<font color='teal'> The intercept is the mean `AdultWeekend` price for all the resorts given the other characteristics. The addition or subtraction of each of the coefficient values in the regression are numeric adjustments applied to the intercept to provide a particular observation's value for the resulting `AdultWeekend` value. Also, because we took the time to scale our x values in the training data, we can compare each of the coeeficients for the features to determine the feature importances. Print the coefficient values from the linear model and sort in descending order to identify the top ten most important features.</font>** 


Hint: make sure to review the absolute value of the coefficients, because the adjustment may be positive or negative, but what we are looking for is the magnitude of impact on our response variable.

In [23]:
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient']).sort_values(by=['Coefficient'], ascending=False).head(12)

,Coefficient
total_chairs,1.119331e+15
fastSixes,9.490325e+14
state_Nevada,6.432944e+14
double,5.983629e+14
surface,5.858615e+14
triple,4.995603e+14
fastEight,4.151681e+14
trams,3.888995e+14
quad,2.845958e+14
state_Rhode Island,2.503039e+14


**<font color='teal'>You should see that the top ten important features are different states. However, the state is not something the managers at the Big Mountain Resort can do anything about. Given that we care more about actionable traits associated with ticket pricing, rebuild the model without the state features and compare the results. </font>**

Hint: Try to construct another model using exactly the steps we followed above. 

#### Model 2

In [24]:
df_new=df.loc[:, ~df.columns.str.contains('state')]

In [25]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         173 non-null    int64  
 1   Unnamed: 0.1       173 non-null    int64  
 2   Name               173 non-null    object 
 3   summit_elev        173 non-null    int64  
 4   vertical_drop      173 non-null    int64  
 5   trams              173 non-null    int64  
 6   fastEight          173 non-null    float64
 7   fastSixes          173 non-null    int64  
 8   fastQuads          173 non-null    int64  
 9   quad               173 non-null    int64  
 10  triple             173 non-null    int64  
 11  double             173 non-null    int64  
 12  surface            173 non-null    int64  
 13  total_chairs       173 non-null    int64  
 14  Runs               173 non-null    float64
 15  TerrainParks       173 non-null    float64
 16  LongestRun_mi      173 non

In [26]:
# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X = df_new.drop(['Name','AdultWeekend'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y = df_new['AdultWeekend'] 

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = preprocessing.StandardScaler().fit(X)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled=scaler.transform(X) 

In [27]:
# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)

In [28]:
lm = linear_model.LinearRegression()
model_2 = lm.fit(X_train,y_train)

In [29]:
y_pred = model_2.predict(X_test)

In [30]:
evs_2 = explained_variance_score(y_test, y_pred)

In [31]:
evs_2

0.7361740918251569

In [32]:
mae_2 = mean_absolute_error(y_test, y_pred)

In [33]:
mae_2

6.885169608640351

In [34]:
pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient']).sort_values(by=['Coefficient'], ascending=False).head(12)

,Coefficient
AdultWeekday,9.173508
summit_elev,3.781364
vertical_drop,2.964957
clusters,2.812584
daysOpenLastYear,1.776544
triple,1.630618
LongestRun_mi,1.500264
quad,1.382402
SkiableTerrain_ac,1.360933
TerrainParks,1.323927


**<font color='teal'> When reviewing our new model coefficients, we see `summit_elev` is now in the number two spot. This is also difficult to change from a management prespective and highly correlated with `base_elev` and `vertical_drop`.  This time, rebuild the model without the state features and without the `summit_elev` and without `base_elev`and compare the results. </font>**

#### Model 3

In [35]:
df_new = df_new.drop(['summit_elev', 'vertical_drop'], axis=1)

In [36]:
# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X = df_new.drop(['Name','AdultWeekend'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y = df_new['AdultWeekend'] 

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = preprocessing.StandardScaler().fit(X)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled=scaler.transform(X) 

In [37]:
# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)

In [38]:
lm = linear_model.LinearRegression()
model_3 = lm.fit(X_train,y_train)

In [39]:
pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient']).sort_values(by=['Coefficient'], ascending=False).head(12)

,Coefficient
AdultWeekday,9.374294
clusters,2.380657
LongestRun_mi,2.133946
averageSnowfall,2.007726
SkiableTerrain_ac,1.669632
triple,1.610729
surface,1.397057
quad,1.221514
daysOpenLastYear,1.171578
double,0.881155


In [40]:
y_pred = model_3.predict(X_test)

In [41]:
evs_3 = explained_variance_score(y_test, y_pred)

In [42]:
evs_3

0.7586494224615249

In [43]:
mae_3 = mean_absolute_error(y_test, y_pred)

In [44]:
mae_3

6.227608557002404

## Identify the Final Model

**<font color='teal'> Review the model performances in the table below and choose the best model for proving insights to Big Mountain management about what features are driving ski resort lift ticket prices. Type your choice in the final markdown cell — you will discuss this selection more in the next step of the guided casptone. </font>**

| Model | Explained Variance| Mean Absolute Error|Features Dropped|
| --- | --- | --- | --- |
| Model 1. | -6.00 |288582161650987.9|-|
| Model 2. | 0.73 | 6.89 |'state'|
| Model 3. | 0.75 | 6.23 |'state','summit_elev','vertical_drop'|

Model Selection: Model 3 